In [45]:
import pandas as pd
import wrangle 
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
df = pd.read_csv('anonymized-curriculum-access.txt', sep=' ', header=None)
df.head(2)

,0,1,2,3,4,5
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61


In [49]:
df = df.rename(columns={0:'date',1:'time',2:'page',3:'user', 4:'cohort',5:'ip' })
df.head()

,date,time,page,user,cohort,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61


# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?


In [50]:
# this is the code that begins to awser question #1
df.page.value_counts().head(10)

page
/                           50313
search/search_index.json    19519
javascript-i                18983
toc                         18297
java-iii                    13733
html-css                    13635
java-ii                     12685
spring                      12524
jquery                      11525
mysql                       11033
Name: count, dtype: int64

In [52]:
# this code gets the awnser to number one entirely
# size allows us to get the counts for two 
q_one = df.groupby(['cohort', 'page']).size().reset_index(name='count')
q_one

,cohort,page,count
0,1.0,/,626
1,1.0,0_Classification_Algorithms,1
2,1.0,2.0_Intro_Stats,1
3,1.0,5.0_Intro_Regression,2
4,1.0,7.0_Intro_Clustering,2
...,...,...,...
13560,139.0,jquery,4
13561,139.0,jquery/essential-methods/attributes-and-css,1
13562,139.0,search/search_index.json,9
13563,139.0,spring,2


In [59]:
top_lessons = q_one.groupby('page')['count'].sum().sort_values(ascending  = False)
top_lessons.head()

page
/                           45854
javascript-i                18203
toc                         17591
search/search_index.json    17534
java-iii                    13166
Name: count, dtype: int64

In [54]:
# can use basketball deep learnign code to break the cohorts apart and comare their top 5 taht way
q_one.sort_values(by='count', ascending=False).head(10)

,cohort,page,count
5922,28.0,/,6340
11247,59.0,/,2980
4528,24.0,/,2101
9731,53.0,/,2038
8643,34.0,/,1967
12221,62.0,toc,1866
3576,22.0,/,1828
7038,28.0,javascript-i,1817
7325,29.0,/,1798
10680,57.0,/,1727


# 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?


In [71]:
#groupby cohort and page to find count of page accesses by each cohort 
access_page_cohort = df.groupby(['cohort', 'page']).size().reset_index(name = 'count')
access_page_cohort

,cohort,page,count
0,1.0,/,626
1,1.0,0_Classification_Algorithms,1
2,1.0,2.0_Intro_Stats,1
3,1.0,5.0_Intro_Regression,2
4,1.0,7.0_Intro_Clustering,2
...,...,...,...
13560,139.0,jquery,4
13561,139.0,jquery/essential-methods/attributes-and-css,1
13562,139.0,search/search_index.json,9
13563,139.0,spring,2


In [74]:
# Identify the top accesses pages by each cohort
top_access = access_page_cohort.loc[access_page_cohort.groupby('cohort')['count'].idxmax()]
top_access

,cohort,page,count
0,1.0,/,626
342,2.0,/,17
386,4.0,/,1
390,5.0,/,1
425,6.0,javascript-ii/es6,10
436,7.0,/,51
579,8.0,/,210
858,9.0,content/html-css,2
861,11.0,/,19
982,12.0,content/html-css,19
